## How to learn Qiskit:

- [https://qiskit.org/learn/](https://qiskit.org/learn/)
- [https://qiskit.org/documentation/](https://qiskit.org/documentation/)

#### Programming SDKs for quantum computers (QSDKs) that exist today:
1. IBM - <b>Qiskit</b> (Python, https://github.com/Qiskit/)
2. Google - <b>Cirq</b> (Python, https://github.com/quantumlib/Cirq)
3. Rigetti - <b>PyQuil</b> (Python, https://github.com/rigetti/pyquil)
4. Microsoft - <b>Microsoft Quantum Development Kit</b> (Q#, https://github.com/microsoft/Quantum)
5. ETH Zurich - <b>Project Q</b> (Python, https://github.com/ProjectQ-Framework/ProjectQ)

#### Good overview publication: https://arxiv.org/pdf/1807.02500.pdf

# Let's learn basics on the Bell's state example...

In [ ]:
# Building a circuit:

from qiskit import QuantumCircuit
from qiskit.providers.aer import QasmSimulator

circuit = QuantumCircuit(2, 2)

circuit.h(0)
circuit.cx(0, 1)
circuit.measure(0, 0) 
circuit.measure(1, 1)

%matplotlib inline
circuit.draw('mpl')

In [ ]:
#Choosing backend and running the circuit

from qiskit import execute

backend = QasmSimulator()

job = execute(circuit, backend, shots=1024)

In [ ]:
#Getting the results and visualizing them

from qiskit.visualization import plot_histogram

result = job.result()
counts = result.get_counts(circuit)
print(counts)

plot_histogram(counts)

Running jobs with remembering individual shots/runs:

In [ ]:
job = execute(circuit, backend, shots=20, memory=True)
data = job.result().get_memory()
print(data)

## Providers and Backends

Provider
- entity that provides access to group of backends
- Qiskit includes 2 providers: `Aer` and `IBMQ`
- implements:
    - `backends()`
    - `get_backend(name)`

Backend
- represents either simulator or real quantum computer
- inherits from `BaseBackend`
- implements:
    - `run(qobj)`
    - `configuration()`
    - `properties()`
    - `name()`
    - `status()`

## Account and token

`IBMQ.save_account('<TOKEN>')`

If account is already saved, it can be overwritten with:

`IBMQ.save_account('<TOKEN>', overwrite=True)`

`IBMQ.stored_account()` - shows what account is stored on the machine

`IBMQ.delete_account()` - deletes currently stored account

More details: https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq

In [ ]:
from qiskit import IBMQ

provider = IBMQ.load_account()

In [ ]:
IBMQ.providers()

In [ ]:
provider.backends()

In [ ]:
from qiskit.tools.monitor import backend_overview, backend_monitor
backend_overview()

In [ ]:
real_backend = provider.get_backend('ibmq_belem')
job_real = execute(circuit, real_backend, shots=1024)
job_real_id=job_real.job_id()
job_real_id

In [ ]:
print(job_real.queue_position())

In [ ]:
from qiskit.tools.monitor import job_monitor

job_monitor(job_real, interval=5)

In [ ]:
print(job_real.status())

`<JobStatus.QUEUED: 'job is queued'>`

`<JobStatus.RUNNING: 'job is actively running'>`

`<JobStatus.DONE: 'job has successfully run'>`

In [ ]:
for job in real_backend.jobs():
    print(str(job.job_id()) + "   " + str(job.creation_date()) + "   " + str(job.status()))

In [ ]:
real_result = real_backend.retrieve_job(job_real_id).result()

In [ ]:
real_counts = real_result.get_counts()

In [ ]:
plot_histogram(real_counts)

In [ ]:
plot_histogram([real_counts, counts], legend=["Real backend", "Simulator"])

## Exercise 1 - coin flip simulation
Write program in Qiskit that will generate N random bit values (0 or 1) using Hadamard gate and quantum simulator.
Hadamard gate puts the qubit (initialized in |0>) in a superposition of |0> and |1> states. Subsequent measurement gives equal probability of getting 0 and 1. N is a parameter which can be set for any number.
We use simulator, which means the numbers are not really random (are pseudo-random), but in subsequent exercise we will use real quantum computer for generating really random numbers.

## Exercise 2 - random numbers generator

Generate 100 random (not pseudo-random ;)) bits.
You do this in the same way as in case of Exercise 1, but use real quantum computer this time

## How to get matrices representing circuits and vectors representing states?

In [ ]:
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(1)
qc.x(0)
qc.h(0)

#Look - no measurement!!!

minus_state = Statevector.from_instruction(qc)
minus_state.draw('latex')

In [ ]:
minus_state.draw('bloch')

In [ ]:
from qiskit.quantum_info.operators import Operator

circuit2 = QuantumCircuit(1, 1)
circuit2.h(0)

U = Operator(circuit2)

# Show the results
print(U.data)

In [ ]:
from qiskit.visualization import array_to_latex
array_to_latex(U.data)

## Exercise 3

Visualize the rotations on Bloch sphere to create $|->$ state

In [ ]:
from qiskit.visualization import visualize_transition

visualize_transition(qc, fpg=40, spg=2, trace=True)

## Now let's create a 3-qubit circuit which is a linear combination of only the following two basis states $|101>$ and $|011>$

In [ ]:
qc3 = QuantumCircuit(3, 3)
qc3.x(0)
qc3.h(1)
qc3.cx(1,2)
qc3.x(2)
#qc.x(2)
qc3.measure([0,1,2], [0,1,2])

backend3 = QasmSimulator()
job3 = execute(qc3, backend3, shots=1024)
result3 = job3.result()
counts3 = result3.get_counts(qc3)
print(counts3)

plot_histogram(counts3)

## Homework

Write qiskit circuit introducing the superposition of the binary representation of first letters of your name and surname.
Write Matplotlib plot visualizing the result.
Do the visualization when run on simulator.
Then run on real hardware limiting to 5 qubits and adding fixed constant qubits later on.

**Send the Jupyter Notebook to pawel.gora@qaif.org**

In [ ]:
# Hint on visualization:

import matplotlib.pyplot as plt
plt.rc('font', family='monospace')
plt.annotate('T', (0.5,0.5), va="center", ha="center", color = (0,0,0,0.1), size = 300)
plt.annotate('S', (0.5,0.5), va="center", ha="center", color = (0,0,0,0.8), size = 300)
plt.axis('off')
plt.show()